In [2]:
from __future__ import division, unicode_literals

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from collections import namedtuple

from ipywidgets import interact, interactive, fixed, interact_manual

import warnings
warnings.filterwarnings('ignore')

BOKEH_TOOLS = 'pan,wheel_zoom,box_zoom,reset,resize,hover'

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
def system_tonescale_segmented(aces= np.logspace(np.log10(1e-5), np.log10(1e5), 1000),
                               xmid=10**0,
                               xmax=10**4,
                               xmin=10**-4,
                               ymid=10**0,
                               ymax=10**4,
                               ymin=10**-4,
                               lo_slope=0.25,
                               hi_slope=0.25,
                               mid_slope=1.5,
                               nKnotsLow=4,
                               nKnotsHigh=4,
                               coefsLow=[],
                               coefsHigh=[]):
    
    oneValue = isinstance(aces,float)

    if oneValue:
        aces = np.array([aces, aces])

    if not coefsLow:
        coefsLow = np.array([0,
                             0,
                             np.log10(ymin) + 0.25 * (np.log10(ymid) - np.log10(ymin)),
                             0,
                             0])

    if not coefsHigh:
        coefsHigh = np.array([0,
                             0,
                             np.log10(ymid) + 0.75 * (np.log10(ymax) - np.log10(ymid)),
                             0,
                             0])


    # Textbook monomial to basis-function conversion matrix
    m = np.matrix('0.5, -1.0,  0.5; -1.0,  1.0,  0.0; 0.5,  0.5,  0.0')

    # B-spline coefficients are are located at the midpoints of each knot interval.

    ## First B-spline
    # Calculate values to be used in application of the B-spline s
    knot_xLow = np.linspace(np.log10(xmin),np.log10(xmid),nKnotsLow)
    knotIncLow = (knot_xLow[-1] - knot_xLow[0]) / (nKnotsLow-1)

    knot_xHigh = np.linspace(np.log10(xmid),np.log10(xmax),nKnotsHigh)
    knotIncHigh = (knot_xHigh[-1] - knot_xHigh[0]) / (nKnotsHigh-1)


    # This block can be commented out if co/kn outputs are not used
    coef_xLow = np.linspace(np.log10(xmin)-0.5 * knotIncLow,np.log10(xmid)+0.5*knotIncLow,nKnotsLow+1)
    coef_xHigh = np.linspace(np.log10(xmid)-0.5 * knotIncHigh,np.log10(xmax)+0.5*knotIncHigh,nKnotsHigh+1)

    coefsLow[0] = (lo_slope * coef_xLow[0]) + ( np.log10(ymin) - lo_slope * knot_xLow[0])
    coefsLow[1] = (lo_slope * coef_xLow[1]) + ( np.log10(ymin) - lo_slope * knot_xLow[0])

    coefsLow[-2] = (mid_slope * coef_xLow[-2]) + ( np.log10(ymid) - mid_slope * knot_xLow[-1])
    coefsLow[-1] = (mid_slope * coef_xLow[-1]) + ( np.log10(ymid) - mid_slope * knot_xLow[-1])

    coefsHigh[0] = (mid_slope * coef_xHigh[0]) + ( np.log10(ymid) - mid_slope * knot_xHigh[0])
    coefsHigh[1] = (mid_slope * coef_xHigh[1]) + ( np.log10(ymid) - mid_slope * knot_xHigh[0])

    coefsHigh[-2]= (hi_slope * coef_xHigh[-2]) + ( np.log10(ymax) - hi_slope * knot_xHigh[-1])
    coefsHigh[-1] = (hi_slope * coef_xHigh[-1]) + ( np.log10(ymax) - hi_slope * knot_xHigh[-1])

    # Parameters calculated from editable parameters:
    # Knot_dens is density of the spline at the knots
    knot_yLow = np.zeros(nKnotsLow)

    for k in np.arange(0, nKnotsLow):
        knot_yLow[k] = (coefsLow[k] + coefsLow[k+1]) / 2

    knot_yHigh = np.zeros(nKnotsHigh)

    for k in np.arange(0, nKnotsHigh):
        knot_yHigh[k] = (coefsHigh[k] + coefsHigh[k+1]) / 2

    # Calculations

    # Check for negatives or zero before taking the log. If negative or zero,
    # set to ACESMIN
    aces[aces <= 0] = 2**-14

    # Allocate an array for the output values
    logOut = np.zeros(np.shape(aces))

    # Take log of linear input
    logAces = np.log10(aces)

    indexLow = ( logAces <= np.log10(xmin));
    indexLowHalf = ((logAces > np.log10(xmin)) & (logAces < np.log10(xmid)))
    indexHighHalf = ((logAces >= np.log10(xmid)) & (logAces < np.log10(xmax)))
    indexHigh = ( logAces >= np.log10(xmax))

    # For values outside the B-spline range, apply linear extension
    logOut[indexLow] = logAces[indexLow] * lo_slope + ( np.log10(ymin) - lo_slope * knot_xLow[0])

    # For typical OUT values apply the B-spline curve.
    if sum(indexLowHalf) > 0:
        knot_coord = ( nKnotsLow - 1) * (logAces[indexLowHalf]-knot_xLow[0]) / (knot_xLow[-1] - knot_xLow[0])
        jLow = knot_coord.astype(int)
        tLow = knot_coord-jLow

        cfLow = np.array(coefsLow[jLow])
        cfLow = np.vstack((cfLow,coefsLow[jLow+1]))
        cfLow = np.vstack((cfLow,coefsLow[jLow+2]))

        monomialsLow = np.array([ tLow * tLow, tLow, np.ones(np.size(tLow))])
        basisLow = np.dot(m,cfLow)
        logOut[indexLowHalf] = np.sum(np.multiply(monomialsLow,basisLow),0)


    # Do the other B-spline for indexHigh
    if sum(indexHighHalf) > 0:
        knot_coord = ( nKnotsHigh - 1) * (logAces[indexHighHalf]-knot_xHigh[0]) / (knot_xHigh[-1] - knot_xHigh[0])
        jHigh = knot_coord.astype(int)
        tHigh = knot_coord-jHigh

        cfHigh = np.array(coefsHigh[jHigh])
        cfHigh = np.vstack((cfHigh,coefsHigh[jHigh+1]))
        cfHigh = np.vstack((cfHigh,coefsHigh[jHigh+2]))

        monomialsHigh = np.array([tHigh * tHigh, tHigh, np.ones(np.size(tHigh))])
        basisHigh = np.dot(m, cfHigh)
        logOut[indexHighHalf] = np.sum(np.multiply(monomialsHigh, basisHigh), 0)

    # For values outside the B-spline range, apply linear extension
    logOut[indexHigh] = logAces[indexHigh] * hi_slope + ( np.log10(ymax) - hi_slope * knot_xHigh[-1])

    kn = {"x": np.hstack((knot_xLow, knot_xHigh)),
          "y": np.hstack((knot_yLow, knot_yHigh))}

    co = {"x": np.hstack((coef_xLow, coef_xHigh)),
          "y": np.hstack((coefsLow,coefsHigh))}

    out = 10**logOut

    if oneValue:
        out = out[0]

    return(out,kn,co)


In [6]:
ACES = np.logspace(np.log10(1e-5), np.log10(1e5), 1000)
XMID=10**0
XMAX=10**4
XMIN=10**-4
YMID=10**0
YMAX=10**4
YMIN=10**-4
LO_SLOPE=0.25
HI_SLOPE=0.25
MID_SLOPE=1.5
NKNOTSLOW=4
NKNOTSHIGH=4
COEFSLOW=[]
COEFSHIGH=[]

def system_tonescale_segmented_interactive(xmid=XMID,
                                           xmax=XMAX,
                                           xmin=XMIN,
                                           ymid=YMID,
                                           ymax=YMAX,
                                           ymin=YMIN,
                                           lo_slope=LO_SLOPE,
                                           hi_slope=HI_SLOPE,
                                           mid_slope=MID_SLOPE,
                                           nKnotsLow=NKNOTSLOW,
                                           nKnotsHigh=NKNOTSHIGH,
                                           coefsLow=COEFSLOW,
                                           coefsHigh=COEFSHIGH):
    
    out,kn,co = system_tonescale_segmented(ACES,xmid,xmax,xmin,ymid,ymax,ymin,
                                           lo_slope,hi_slope,mid_slope,
                                           nKnotsLow,nKnotsHigh,coefsLow,coefsHigh)
    
    plot = figure(tools=BOKEH_TOOLS, x_range=(-5, 5), y_range=(-5, 5))
    sts_line = plot.line(np.log10(ACES), np.log10(out))
    
    handle = show(plot, notebook_handle=True)
    
    return {'sts_line': sts_line, 'handle': handle}

sts_t_o_i = system_tonescale_segmented_interactive(xmid=XMID,
                                           xmax=XMAX,
                                           xmin=XMIN,
                                           ymid=YMID,
                                           ymax=YMAX,
                                           ymin=YMIN,
                                           lo_slope=LO_SLOPE,
                                           hi_slope=HI_SLOPE,
                                           mid_slope=MID_SLOPE,
                                           nKnotsLow=NKNOTSLOW,
                                           nKnotsHigh=NKNOTSHIGH,
                                           coefsLow=COEFSLOW,
                                           coefsHigh=COEFSHIGH)

def update_sts_t_o_i(xmid=XMID):
#                      xmax=XMAX,
#                      xmin=XMIN,
#                      ymid=YMID,
#                      ymax=YMAX,
#                      ymin=YMIN,
#                      lo_slope=LO_SLOPE,
#                      hi_slope=HI_SLOPE,
#                      mid_slope=MID_SLOPE,
#                      nKnotsLow=NKNOTSLOW,
#                      nKnotsHigh=NKNOTSHIGH,
#                      coefsLow=COEFSLOW,
#                      coefsHigh=COEFSHIGH):
    
    sts_t_o_i['sts_line'].data_source.data['y'] = system_tonescale_segmented(xmid)
#     xmax,xmin,ymid,ymax,ymin,
#     lo_slope,hi_slope,mid_slope,
#     nKnotsLow,nKnotsHigh,coefsLow,coefsHigh)
    push_notebook(handle=sts_t_o_i['handle']) 

In [8]:
# interact(update_UE4_t_o_i, film_slope=(0, 2, 0.001), film_toe=(0, 2, 0.001), film_shoulder=(0, 2, 0.001), film_black_clip=(0, 2, 0.001), film_white_clip=(0, 2, 0.001), in_match=(0, 2, 0.001), out_match=(0, 2, 0.001))

interact(update_sts_t_o_i, xmid=(10**-2, 10**1, 0.1))


# xmid=XMID,
#                                            xmax=XMAX,
#                                            xmin=XMIN,
#                                            ymid=YMID,
#                                            ymax=YMAX,
#                                            ymin=YMIN,
#                                            lo_slope=LO_SLOPE,
#                                            hi_slope=HI_SLOPE,
#                                            mid_slope=MID_SLOPE,
#                                            nKnotsLow=NKNOTSLOW,
#                                            nKnotsHigh=NKNOTSHIGH,
#                                            coefsLow=COEFSLOW,
#                                            coefsHigh=COEFSHIGH